In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import shutil
import os

from utils import  parse_contributors, parse_leitzahl, get_missing_leitzahls

from constants import CKONSORG_COLUMNS, LEITZAHL_COLUMNS_EXT, PUBLICATION_COLUMNS_EXT, CKONSORG_TABLE, INPUT_DATA_PATH, OUT_BCP_PATH, ALL_COLUMNS_SORTED, PUBLICATION_COLUMNS, PUBLICATION_TABLE, AUTHORSHIP_COLUMNS, AUTHORSHIP_TABLE, MASTER_TABLE, LEITZAHL_COLUMNS, LEITZHAL_TABLE
from db.db_importer import dump_table, change_column_type_to_int


INFO - Connected to DB: <bound method PSQL_DB.__repr__ of Postgres('biblioowner', <password hidden>, 'id-hdb-psgr-cp46.ethz.ch', '5432', 'bibliometrics')>


# 1. Download

Download the data per year from Research Collection API with a developer APIKey (see `download_pipeline.ipynb`)

# 2. Data parsing 

1. Read and load the data in pandas Dataframes
2. Extract and parse different columns as separated entities from the dataframe
3. Write these entities in bcp files
4. Run bash scripts to import bcp files in postgres database (**REQUIRED**: *sql table creation scripts already executed*)

## 2.1 Load data in pandas dataframe

In [2]:
import logging
from pathlib import Path
from typing import List

INPUT_DATA_PATH = '/home/bibliometric/data/research_collection/input/'

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def bulk_load_df() -> pd.DataFrame:
    """
    Load multiple CSV files from the input directory into a single DataFrame.

    Returns:
        pd.DataFrame: A concatenated DataFrame containing data from all CSV files.
    """
    df_list: List[pd.DataFrame] = []

    for file_name in sorted(Path(INPUT_DATA_PATH).iterdir()):
        if file_name.is_file() and file_name.suffix == '.csv':
            try:
                logging.info(f'Reading file: {file_name}')
                df_list.append(pd.read_csv(file_name, index_col=False, low_memory=False, dtype=str))
            except pd.errors.EmptyDataError:
                logging.error(f'EmptyDataError - {file_name} is empty and will be skipped.')
            except pd.errors.ParserError:
                logging.error(f'ParserError - {file_name} is malformed and will be skipped.')
            except Exception as error:
                logging.error(f'Error reading {file_name} => {error}')
    
    if df_list:
        logging.info('Concatenating dataframes')
        return pd.concat(df_list, ignore_index=True)
    else:
        logging.warning('No dataframes to concatenate, returning an empty DataFrame.')
        return pd.DataFrame()

## 2.2-4 Extract, parse, migrate to DB

Extract the relative columns used in the SQL tables from the dataframe and parse them if needed. 

Write the dataframe in bcp files ready to be imported in Postgres

### 2.x.1 Extract publications

In [3]:
def parse_publication(df: pd.DataFrame, reload: bool = True, sep:str = '\t'):
	BCP_FILE = 'publications.bcp'
	bcp_file_path = OUT_BCP_PATH+BCP_FILE
	if reload:
		publications_df = df[df.columns.intersection(set(PUBLICATION_COLUMNS))]
		publications_df = publications_df.reindex(columns=PUBLICATION_COLUMNS)
		publications_df['rc_year'] = publications_df['dc_date_issued'].str.extract(r'([0-9][0-9][0-9][0-9])', expand=True)
		publications_df.to_csv(bcp_file_path, sep=sep, index=False, index_label='\t', header=False)
	print(f'== INFO - DONE - Wrote publications in file {bcp_file_path=}')
	#dump_table(table_name=PUBLICATION_TABLE, bcp_file=bcp_file_path, columns=PUBLICATION_COLUMNS)
	tryAgain = -1
	count = 0
	skipped_file_path = OUT_BCP_PATH+'skipped_pub.bcp'
	while tryAgain != 0:
		print(f"== START Try Again {count=}")
		tryAgain = int(dump_table(table_name=PUBLICATION_TABLE, bcp_file=bcp_file_path, columns=PUBLICATION_COLUMNS, sep=sep))
		print(f'{tryAgain=}')
		if tryAgain > 0:
			count += 1
			skipped_row = ''
			with open( bcp_file_path, "r+" ) as f:
				lines = f.readlines()
				f.seek(0)
				skipped_row = lines.pop( tryAgain - 1 )
				print(f"INFO - Removed {skipped_row=}")
				f.truncate()
				f.writelines( lines ) 
			with open(skipped_file_path, "a" ) as f2:
				f2.write(skipped_row)
	else:
		print("== Try Again DONE")

### 2.x.2 Extract contributors

In [4]:
def parse_autorship(df: pd.DataFrame):
	BCP_FILE = 'authorship.bcp'
	bcp_file_path = OUT_BCP_PATH+BCP_FILE
	#global_authors_set = set()
	contributors_set, local_authorship_list = parse_contributors(df)
	#global_authors_set.update(contributors_set)
	#pd.DataFrame(global_authors_set).to_csv('./data/output/author.bcp', sep='\t', index=False, index_label='\t', header=False)
	global_authorship_df = pd.DataFrame(local_authorship_list, columns=AUTHORSHIP_COLUMNS)
	global_authorship_df.to_csv(bcp_file_path, sep='\t', index=False, index_label='\t', header=False)
	print(f'== INFO - DONE - Wrote authorships in file {bcp_file_path=}')
	#dump_table(table_name=AUTHORSHIP_TABLE,  bcp_file=bcp_file_path, columns=AUTHORSHIP_COLUMNS)
	tryAgain = -1
	count = 0
	skipped_file_path = OUT_BCP_PATH+'skipped_aut.bcp'
	while tryAgain != 0:
		print(f"== START Try Again {count=}")
		tryAgain = int(dump_table(table_name=AUTHORSHIP_TABLE,  bcp_file=bcp_file_path, columns=AUTHORSHIP_COLUMNS))
		print(f'{tryAgain=}')
		if tryAgain > 0:
			count += 1
			skipped_row = ''
			with open( bcp_file_path, "r+" ) as f:
				lines = f.readlines()
				f.seek(0)
				skipped_row = lines.pop( tryAgain - 1 )
				print(f"INFO - Removed {skipped_row=}")
				f.truncate()
				f.writelines( lines ) 
			with open(skipped_file_path, "a" ) as f2:
				f2.write(skipped_row)
	else:
		print("== Try Again DONE")

In [ ]:
def parse_master(df: pd.DataFrame, reload: bool = True, drop_table: bool = False, sep: str = '\t'):
    BCP_FILE = 'master.bcp'
    bcp_file_path = OUT_BCP_PATH + BCP_FILE
    skipped_file_path = OUT_BCP_PATH + 'skipped.bcp'

    if reload:
        df.to_csv(bcp_file_path, sep=sep, index=False, header=False)
        print(f'== INFO - DONE - Wrote master table to file {bcp_file_path}')

    try_again = -1
    count = 0

    while try_again != 0:
        print(f"== START Try Again {count}")
        try_again = dump_table(table_name=MASTER_TABLE, bcp_file=bcp_file_path, columns=df.columns.to_list(), drop_table=drop_table, sep=sep)
        if try_again > 0:
            count += 1
            with open(bcp_file_path, "r+") as f:
                lines = f.readlines()
                f.seek(0)
                skipped_row = lines.pop(try_again - 1)
                print(f"INFO - Removed row: {skipped_row.strip()}")
                f.truncate()
                f.writelines(lines)
            with open(skipped_file_path, "a") as f2:
                f2.write(skipped_row)
        elif try_again == -1:
            print("ERROR - Unrecoverable error during dumping table")
            break
    else:
        print("== Try Again DONE")

""" def parse_master(df: pd.DataFrame, reload: bool = True, drop_table: bool = False, sep:str = '\t'):
	BCP_FILE = 'master.bcp'
	bcp_file_path = OUT_BCP_PATH+BCP_FILE
	skipped_file_path = OUT_BCP_PATH+'skipped.bcp'
    
    if reload:
		#df.to_csv('master.csv')
		df.to_csv(bcp_file_path, sep=sep, index=False, index_label='\t', header=False)
		print(f'== INFO - DONE - Wrote master table in file {bcp_file_path=}')
	#dump_table(table_name=MASTER_TABLE, bcp_file=bcp_file_path, columns=df.columns.to_list())
	tryAgain = -1
	count = 0
	
	while tryAgain != 0:
		print(f"== START Try Again {count=}")
		tryAgain = int(dump_table(table_name=MASTER_TABLE, bcp_file=bcp_file_path, columns=df.columns.to_list(), drop_table=drop_table, sep=sep))
		print(f'{tryAgain=}')
		if tryAgain > 0:
			count += 1
			skipped_row = ''
			with open( bcp_file_path, "r+" ) as f:
				lines = f.readlines()
				f.seek(0)
				skipped_row = lines.pop( tryAgain - 1 )
				print(f"INFO - Removed {skipped_row=}")
				f.truncate()
				f.writelines( lines ) 
			with open(skipped_file_path, "a" ) as f2:
				f2.write(skipped_row)
	else:
		print("== Try Again DONE") """

In [6]:
import pandas as pd
from constants import PATH_TO_CKONSORG_DATA, CKONSORG_COLUMNS
from utils import get_latest_ckonsorg_filename

def parse_ckonsorg_data(dump_table:bool = False):

	ckonsorg_file_name = get_latest_ckonsorg_filename()

	df = pd.read_csv(PATH_TO_CKONSORG_DATA + ckonsorg_file_name, index_col=False, low_memory=False, dtype=str)
	
	df.columns = [x.lower() for x in df.columns]

	#take the subset needed
	ckonsorg_df = df[CKONSORG_COLUMNS]

	# Identify columns that start with 'LZ'
	lz_columns = [col for col in ckonsorg_df.columns if col.startswith('lz')]

	# Replace 'T' with '0' in the filtered columns
	ckonsorg_df.loc[:, lz_columns] = ckonsorg_df.loc[:, lz_columns].applymap(lambda x: x.replace('T', '0'))
	ckonsorg_df['vondat'] = ckonsorg_df['vondat'].str.extract(r'([0-9]{4})', expand=True)
	ckonsorg_df['bisdat'] = ckonsorg_df['bisdat'].str.extract(r'([0-9]{4})', expand=True)
	ckonsorg_df.head(10)

	ckonsorg_bcp_file = 'ckonsorg.bcp'
	bcp_file_path = PATH_TO_CKONSORG_DATA + ckonsorg_bcp_file
	ckonsorg_df.to_csv(bcp_file_path, sep='\t', index=False, header=False)
	print('Created BCP file: ', bcp_file_path)
	if dump_table:
		dump_table(table_name=CKONSORG_TABLE, bcp_file=bcp_file_path, columns=CKONSORG_COLUMNS, drop_table=True)

In [7]:
def parse_skipped(skipped_file: str = 'skipped_pub.bcp', 
				  error_file: str = 'error.bcp', 
				  bcp_file:str = 'publications.bcp', 
				  drop_table: bool = True, 
				  columns: list = ALL_COLUMNS_SORTED, 
				  table_name: str = MASTER_TABLE,
				  sep: str = '\t'):
	SKIPPED_PATH = OUT_BCP_PATH + skipped_file
	ERROR_PATH = OUT_BCP_PATH + error_file
	BCP_PATH = OUT_BCP_PATH + bcp_file
	tryAgain = -1
	count = 0
	while tryAgain != 0:
		print(f"== START Try Again {count=}")
		tryAgain = int(dump_table(table_name=table_name, bcp_file=SKIPPED_PATH, columns=columns, drop_table=drop_table, sep=sep))
		#print(f'{tryAgain=}')
		if tryAgain > 0:
			count += 1
			skipped_row = ''
			with open( SKIPPED_PATH, "r+" ) as f:
				lines = f.readlines()
				f.seek(0)
				skipped_row = lines.pop( tryAgain - 1 )
				print(f"INFO - Removed {skipped_row=}")
				f.truncate()
				f.writelines( lines ) 
			with open(ERROR_PATH, "a" ) as f2:
				f2.write(skipped_row)
	else:
		print("== Try Again DONE")
		shutil.copyfileobj(open(SKIPPED_PATH, 'rb'), open(BCP_PATH, 'ab'))

In [8]:
import os
import pandas as pd
import logging
from pathlib import Path
from typing import List, Optional

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

OUT_BCP_PATH2 = Path(OUT_BCP_PATH)

def parse_and_dump(
    df: pd.DataFrame, 
    bcp_file_name: str, 
    table_name: str, 
    columns: Optional[List[str]] = None, 
    reload: bool = True, 
    drop_table: bool = False, 
    sep: str = '\t',
    extract_year: bool = False
) -> None:
    """
    Generic method to parse DataFrame and dump to a file with error handling.

    Parameters:
        df (pd.DataFrame): DataFrame to be processed.
        bcp_file_name (str): Name of the BCP file.
        table_name (str): Name of the table to dump data.
        columns (Optional[List[str]]): List of columns to reindex the DataFrame.
        reload (bool): Flag to reload data.
        drop_table (bool): Flag to drop the table before dumping.
        sep (str): Separator for the CSV file.
        extract_year (bool): Flag to extract the year from 'dc_date_issued' column.
    """
    bcp_file_path = OUT_BCP_PATH2 / bcp_file_name
    skipped_file_path = OUT_BCP_PATH2 / f'skipped_{bcp_file_name}'

    if reload:
        if columns:
            df = df[df.columns.intersection(set(columns))]
            df = df.reindex(columns=columns)
        if extract_year:
            df['rc_year'] = df['dc_date_issued'].str.extract(r'([0-9]{4})', expand=True)
        df.to_csv(bcp_file_path, sep=sep, index=False, index_label='\t', header=False)
        logging.info(f'== INFO - DONE - Wrote data to file {bcp_file_path}')

    try_again = -1
    count = 0

    while try_again != 0:
        logging.info(f"== START Try Again {count}")
        try_again = dump_table(table_name=table_name, bcp_file=bcp_file_path, columns=columns or df.columns.to_list(), drop_table=drop_table, sep=sep)
        logging.info(f'{try_again=}')
        if try_again > 0:
            count += 1
            with open(bcp_file_path, "r+") as f:
                lines = f.readlines()
                f.seek(0)
                skipped_row = lines.pop(try_again - 1)
                logging.info(f"INFO - Removed row: {skipped_row.strip()}")
                f.truncate()
                f.writelines(lines)
            with open(skipped_file_path, "a") as f2:
                f2.write(skipped_row)
        elif try_again == -1:
            logging.error("ERROR - Unrecoverable error during dumping table")
            break
    else:
        logging.info("== Try Again DONE")

# 5. Run

## NOTE
probably in this step a few publications are lost in the `count()` because they not contain the `dc_identifier_uri` and no `handle_id` can be extracted.

In [9]:
# thse steps can be done year by year or as a one big dataset
big_df = bulk_load_df()
reordered_df = big_df.reindex(columns=ALL_COLUMNS_SORTED)
print(big_df.info())
big_df['ethz_size'] = big_df['ethz_size'].apply(lambda size: size.replace('\t', " ") if pd.notna(size) else size)
big_df['handle_id'] = big_df['dc_identifier_uri'].apply(lambda uri: uri.split("/")[-1] if pd.notna(uri) else uri)
big_df['dc_title'] = big_df['dc_title'].apply(lambda title: title.replace('\\', "") if pd.notna(title) else title)

reordered_df = big_df.reindex(columns=ALL_COLUMNS_SORTED)
print(reordered_df.duplicated(subset=['handle_id']).value_counts())
#print(reordered_df.duplicated(subset=['handle_id']))
reordered_df.info()

2024-08-21 17:26:44,541 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1905_metadata.csv
2024-08-21 17:26:44,551 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1909_metadata.csv
2024-08-21 17:26:44,556 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1910_metadata.csv
2024-08-21 17:26:44,560 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1911_metadata.csv
2024-08-21 17:26:44,564 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1912_metadata.csv
2024-08-21 17:26:44,583 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1913_metadata.csv
2024-08-21 17:26:44,594 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1914_metadata.csv
2024-08-21 17:26:44,599 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1915_metadata.csv


2024-08-21 17:26:44,608 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1916_metadata.csv
2024-08-21 17:26:44,627 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1917_metadata.csv
2024-08-21 17:26:44,639 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1918_metadata.csv
2024-08-21 17:26:44,645 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1919_metadata.csv
2024-08-21 17:26:44,657 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1920_metadata.csv
2024-08-21 17:26:44,662 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1921_metadata.csv
2024-08-21 17:26:44,667 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1922_metadata.csv
2024-08-21 17:26:44,672 - INFO - Reading file: /home/bibliometric/data/research_collection/input/1923_metadata.csv
2024-08-21 17:26:44,677 - INFO - Reading file: /home/bibliometric/data/research_

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256122 entries, 0 to 256121
Columns: 134 entries, dc_contributor_author to ethz_date_openbisupload
dtypes: object(134)
memory usage: 261.8+ MB
None


/tmp/ipykernel_1573717/1609896663.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  big_df['handle_id'] = big_df['dc_identifier_uri'].apply(lambda uri: uri.split("/")[-1] if pd.notna(uri) else uri)


False    256120
True          2
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256122 entries, 0 to 256121
Columns: 135 entries, dc_contributor to rc_item_id
dtypes: object(135)
memory usage: 263.8+ MB


### NOTE: 
1. apparently looks like there were duplicates but checking those are false duplicates because 2 records are `NaN` and one has an older `handle_id`
2. checks for `NaN` 

In [10]:
#duplicated_rows = reordered_df[reordered_df.duplicated(subset=['handle_id'], keep=False)]
#duplicated_rows
#null_rows = reordered_df.loc[reordered_df['dc_identifier_uri'].isnull() ] #| reordered_df['dc_identifier_doi'].isnull()]
#null_rows

In [11]:
#full_leitzhal_df = reordered_df.dropna(subset=['ethz_leitzahl'])[['handle_id', 'rc_item_id','ethz_leitzahl', 'ethz_leitzahlidentifiers', 'ethz_leitzahlidentifiers_certified']]
#parse_leitzahl(full_leitzhal_df)

In [12]:
#parse_and_dump(reordered_df, 'publications.bcp', PUBLICATION_TABLE, PUBLICATION_COLUMNS, extract_year=True, drop_table=True)

In [13]:
#parse_and_dump(reordered_df, 'authorship.bcp', AUTHORSHIP_TABLE, AUTHORSHIP_COLUMNS, drop_table=True)

In [14]:
#parse_and_dump(reordered_df, 'master.bcp', MASTER_TABLE, drop_table=True)

In [15]:
#parse_autorship(reordered_df)

In [16]:
#parse_publication(reordered_df, True, sep='\t')

In [17]:
#change_column_type_to_int(PUBLICATION_TABLE, "rc_year")

Correct the publications with errors (usually there are tabs in titles or descriptions or other text fields) and run parse_skipped for publications

In [18]:
#from constants import PUBLICATION_COLUMNS
#parse_skipped(skipped_file = 'skipped_pub.bcp', error_file = 'error.bcp', bcp_file = 'publications.bcp', drop_table=False, columns=PUBLICATION_COLUMNS, table_name=PUBLICATION_TABLE, sep = '\t')

# !!! RUN THIS MANUALLY after the creation of publication table

change year from VARCHAR to INTEGER

```sql
ALTER TABLE "RCPublication"
ALTER COLUMN rc_year TYPE INT USING (cast ( coalesce( nullif( trim(rc_year), '' ), '0' ) as integer ))
```

When running parse_master the first time select reload=True, then after correcting all records with errors with the script in the next section, relaunch parse_master with reload=False to avoid overwriting the corrected bcp file.

In [19]:
#parse_master(df=reordered_df, reload=True, drop_table=True, sep='\t')

Run after "parse_master(big_df, True, '@')" in order to replace all the @ in string that cause errors when writing to the DB with sep=@

In [20]:
#parse_skipped(skipped_file = 'skipped.bcp', error_file = 'error.bcp', bcp_file = 'master.bcp', drop_table = False, columns= ALL_COLUMNS_SORTED, table_name = MASTER_TABLE, sep = '\t')

In [21]:
#parse_ckonsorg_data(dump_table=True)

# !!! RUN THIS MANUALLY after the creation of ckonsorg table

```sql
ALTER TABLE "RCckonsorg"
ALTER COLUMN vondat TYPE INT USING (cast ( coalesce( nullif( trim(vondat), '' ), '0' ) as integer ))
```

```sql
ALTER TABLE "RCckonsorg"
ALTER COLUMN bisdat TYPE INT USING (cast ( coalesce( nullif( trim(bisdat), '' ), '0' ) as integer ))
```

In [22]:
#ckonsorg_bcp_path = '/home/bibliometric/data/research_collection/ckonsorg/ckonsorg.bcp'
#ckonsorg_df = pd.read_csv(ckonsorg_bcp_path, sep='\t', names=CKONSORG_COLUMNS, index_col=False, low_memory=False, dtype=str)
#ckonsorg_df

In [23]:
#leitzahl_bcp_path = '/home/bibliometric/data/research_collection/bcp_out/leitzhal.bcp'
#full_stored_leitzhal_df = pd.read_csv(leitzahl_bcp_path, sep='\t', names=LEITZAHL_COLUMNS, index_col=False, low_memory=False, dtype=str)
#full_stored_leitzhal_df

In [24]:
#lz_df_w_missing_lz = get_missing_leitzahls(ckonsorg_df, full_stored_leitzhal_df)

In [25]:
path_to_update_lz_bcp = '/home/bibliometric/data/research_collection/bcp_out/ckonsorg_updated_leitzahl.bcp'
#lz_df_w_missing_lz.to_csv(path_to_update_lz_bcp, sep='\t', index=False, index_label='\t', header=False)
#dump_table(LEITZHAL_TABLE, path_to_update_lz_bcp, LEITZAHL_COLUMNS_EXT)

# -------------------------------- TESTs -------------------------------------

In [26]:
full_lz_df = pd.read_csv(path_to_update_lz_bcp, sep='\t', names=LEITZAHL_COLUMNS_EXT, index_col=False, low_memory=False, dtype=str)
full_lz_df

,handle_id,rc_item_id,ou_code,ou_name,is_leaf,is_certified,lz_lv_match
0,147275,172054,00007,T-Departemente,False,NaN,lz70
1,147275,172054,00012,T-Lehre + For.,False,NaN,lz80
2,147275,172054,00002,T-ETH Zürich,False,NaN,lz90
3,55989,442093,02610,T-Verkehrspl.Transp.,False,NaN,lz50
4,55989,442093,02115,"T-Bau,Umw.u.Geomatik",False,NaN,lz60
...,...,...,...,...,...,...,...
1436313,679485,551405,00012,Lehre und Forschung,False,NaN,lz80
1436314,679485,551405,00007,Departemente,False,NaN,lz70
1436315,679485,551405,02140,Dep. Inf.technologie und Elektrotechnik / Dep....,False,NaN,lz60
1436316,679485,551405,02636,Institut fr Integrierte Systeme / Integrated S...,False,NaN,lz50


In [27]:
publications_bcp_path = '/home/bibliometric/data/research_collection/bcp_out/publications.bcp'

publications_df = pd.read_csv(publications_bcp_path, sep='\t', names=PUBLICATION_COLUMNS, index_col=False, low_memory=False, dtype=str)
publications_df

,handle_id,rc_item_id,dc_type,ethz_eth,ethz_leitzahl,ethz_leitzahl_certified,dc_title,dc_date_issued,rc_year,dc_identifier_uri,ethz_source,dc_identifier_doi,dc_identifier_other
0,139872,139839,Doctoral Thesis,no,NaN,NaN,Eine neue Bestimmung der Molekldimensionen,1905,1905,http://hdl.handle.net/20.500.11850/139872,ECOL,10.3929/ethz-a-000565688,NaN
1,132784,132751,Doctoral Thesis,yes,NaN,NaN,Zur Kenntnis der Chinonimine und der Chinone,1909,1909,http://hdl.handle.net/20.500.11850/132784,ECOL,10.3929/ethz-a-000090494,NaN
2,131903,131870,Doctoral Thesis,yes,NaN,NaN,Ueber Anilinschwarz,1909,1909,http://hdl.handle.net/20.500.11850/131903,ECOL,10.3929/ethz-a-000088717,NaN
3,132465,132432,Doctoral Thesis,yes,NaN,NaN,Ueber den Abbau von Chlorophyll durch Alkalien,1909,1909,http://hdl.handle.net/20.500.11850/132465,ECOL,10.3929/ethz-a-000089765,NaN
4,132925,132892,Doctoral Thesis,yes,NaN,NaN,Beitrge zur Kenntnis pharmazeutisch verwendete...,1909,1909,http://hdl.handle.net/20.500.11850/132925,ECOL,10.3929/ethz-a-000090766,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
266033,680545,552299,Conference Paper,yes,NaN,NaN,Dynamic O(Arboricity) Coloring in Polylogarith...,2024-06,2024,http://hdl.handle.net/20.500.11850/680545,SCOPUS,10.3929/ethz-b-000680545,10.1145/3618260.3649782
266034,680621,552302,Working Paper,yes,ETH Zrich::00002 - ETH Zrich::00012 - Lehre un...,NaN,Beyond FITT - How Density Can Improve the Unde...,2024-05-21,2024,http://hdl.handle.net/20.500.11850/680621,FORM,10.3929/ethz-b-000680621,10.51224/SRXIV.411
266035,680667,552304,Other Publication,yes,ETH Zrich::00002 - ETH Zrich::00012 - Lehre un...,NaN,A Rearmament Challenge: The Example of Gunpowder,2024-07-05,2024,http://hdl.handle.net/20.500.11850/680667,FORM,NaN,NaN
266036,680073,551880,Journal Article,yes,ETH Zrich::00002 - ETH Zrich::00012 - Lehre un...,ETH Zrich::00002 - ETH Zrich::00012 - Lehre un...,Electric-field activating on-surface tailored ...,2025-01-19,2025,http://hdl.handle.net/20.500.11850/680073,FORM,NaN,10.1016/j.seppur.2024.128291


In [31]:
existing_ou_codes = set(zip(full_lz_df['handle_id'], full_lz_df['rc_item_id']))

# Filter full_lz_df to include only the relevant handle_id and rc_item_id pairs
filtered_lz_df = full_lz_df[full_lz_df.set_index(['handle_id', 'rc_item_id']).index.isin(existing_ou_codes)]

# Group by handle_id and rc_item_id and aggregate ou_code into lists
grouped_lz_df = filtered_lz_df.groupby(['handle_id', 'rc_item_id'])['ou_code'].agg(set).reset_index()

# Merge the aggregated lists back to publications_df
merged_df = publications_df.merge(grouped_lz_df, on=['handle_id', 'rc_item_id'], how='left')

# Add the resulting lists of ou_code to a new column in publications_df
publications_df['pub_lz'] = merged_df['ou_code'].where(
    merged_df[['handle_id', 'rc_item_id']].apply(tuple, axis=1).isin(existing_ou_codes),
    None
)

In [33]:
BCP_FILE = '/home/bibliometric/data/research_collection/bcp_out/publications_extended.bcp'
#bcp_file_path = OUT_BCP_PATH+BCP_FILE
publications_df.to_csv(BCP_FILE, sep='\t', index=False, index_label='\t', header=False)
dump_table(PUBLICATION_TABLE, BCP_FILE, PUBLICATION_COLUMNS_EXT)

== INFO - Executed table creation for RCPublication
== INFO - cursor and connection CLOSED.


0

In [ ]:
stop = get_error()

NameError: name 'get_error' is not defined

In [ ]:
# Identify the 'lz' columns in ckonsorg_df
lz_columns = [col for col in ckonsorg_df.columns if col.startswith('lz')]

# Flatten the ckonsorg_df: Create a DataFrame with 'ou_code', 'lz_match', and 'source_index' columns
flat_df = pd.DataFrame()

for lz_col in lz_columns:
    temp_df = ckonsorg_df[[lz_col]].copy()
    temp_df['lz_match'] = lz_col
    #temp_df['source_index'] = temp_df.index  # Capture the source index
    temp_df = temp_df.rename(columns={lz_col: 'ou_code'})
    flat_df = pd.concat([flat_df, temp_df])

# Drop duplicates to avoid redundant merges
flat_df = flat_df.drop_duplicates(subset=['ou_code'])

# Merge leitzahl_df with flat_df on 'ou_code'
leitzahl_df = full_stored_leitzhal_df.merge(flat_df, on='ou_code', how='left')

In [ ]:
ck_columns = {'lz90':'na90','lz80':'na80','lz70':'na70','lz60':'na60','lz50':'na50','lz40':'na40','lz30':'na30','lz20':'na20'}
ck_keys_list = list(ck_columns).index('lz50')

In [ ]:
leitzahl_row_to_add = []

In [ ]:
# Set-based lookup for faster membership checking
existing_ou_codes = set(zip(leitzahl_df['handle_id'], leitzahl_df['rc_item_id'], leitzahl_df['ou_code']))

filtered_leitzahl_df = leitzahl_df[leitzahl_df.is_leaf == True]

for idx, leitzahl_row in filtered_leitzahl_df.iterrows():
	#print(idx, leitzahl_row)
	ckonsorg_row = ckonsorg_df[ckonsorg_df[leitzahl_row.lz_match].str.contains(leitzahl_row.ou_code)]
	#print('----------- ckonsorg_row')
	#print(ckonsorg_row)
	lz_match_idx = list(ck_columns).index(leitzahl_row.lz_match)
	for i in range(1, lz_match_idx+1):
		ck_parent_key = list(ck_columns)[lz_match_idx-i]
		ck_parent_value = ckonsorg_row[ck_parent_key].values[0]
		#print('-------------------------')
		#print("ck_parent_key = ", ck_parent_key)
		#print('ckonsorg_row[ck_parent_key].values = ', ckonsorg_row[ck_parent_key].values[0])
		#if (not ((leitzahl_df['handle_id'] == leitzahl_row.handle_id) & (leitzahl_df['rc_item_id'] == leitzahl_row.rc_item_id) & (leitzahl_df['ou_code'] == ck_parent_value)).any()):
		if (leitzahl_row.handle_id, leitzahl_row.rc_item_id, ck_parent_value) not in existing_ou_codes:
			#print('isPresent: ', ((leitzahl_df['handle_id'] == leitzahl_row.handle_id) & (leitzahl_df['rc_item_id'] == leitzahl_row.rc_item_id) & (leitzahl_df['ou_code'] == ck_parent_value)).any())
			leitzahl_row_to_add.append({
				'handle_id':leitzahl_row.handle_id, 
				'rc_item_id': leitzahl_row.rc_item_id, 
				'ou_code': ck_parent_value,
				'ou_name': ckonsorg_row[ck_columns.get(ck_parent_key)].values[0],
				'is_leaf':	False,
				'is_certified': None,
				'lz_match':	ck_parent_key})
			existing_ou_codes.add((leitzahl_row.handle_id, leitzahl_row.rc_item_id, ck_parent_value))
	""" if idx > 100:
		break """


In [ ]:
leitzahl_row_to_add

In [ ]:
leitzahl_df

In [ ]:
path_to_update_lz_bcp = '/home/bibliometric/mihai_test/bibliometrics/libbiblio/sources/rc/ckonsorg_updated_leitzahl.bcp'
df3 = pd.concat([pd.DataFrame(leitzahl_row_to_add), leitzahl_df], ignore_index=True)
df3.to_csv("./new_leit.bcp", sep='\t', index=False, index_label='\t', header=False)

In [ ]:

dump_table(LEITZHAL_TABLE, path_to_update_lz_bcp, LEITZAHL_COLUMNS_EXT)

In [ ]:
df3.value_counts('lz_match')

In [ ]:
leitzahl_df.value_counts('lz_match')